# Large-Scale Image Processing On Your Laptop (and elsewhere)

### ImageXD 2017 Tutorial

---
### Goals of this workshop:

* get acquanted with the Python [Dask](http://dask.pydata.org/en/latest/) Library
* learn how to execute basic operations on large images which cannot fit in RAM
* learn about the concepts of lazy evaluation and task scheduling graphs
* learn how to work with [Dask Arrays](http://dask.pydata.org/en/latest/array.html)
* learn how to work with [Dask Delayed](http://dask.pydata.org/en/latest/delayed.html)

---
### Motivation

What is this tutorial about? Researchers across domains get overloaded with image data which their traditional processing workflows are incapable to handle. Usually they are faced with two possible options: 
* move the processing to large machines/clusters
* modify their methods to access the image data only pieces at a time.

<!---* Mechanical Engineering - microbubbles
* Oceanography - sonar, video
* Neuroscience - calcium imaging, fMRI
* etc--->


Scientists like to test out things on their laptops, and later move to clusters, without having to modify their code a lot.

[Dask](http://dask.pydata.org/en/latest/) is a Python Library which makes this possible: 
* can perform computations on images which cannot fit into RAM
* has interface similar to `numpy` and `scipy`
* the same code used on your laptop can be run on a distributed cluster


We will be working with the cells data. So set the path to the location of the cells folder.

In [1]:
path = 'cells/'

In [2]:
# some preliminary imports
import numpy as np
#%matplotlib 'Qt4Agg'
%matplotlib inline
import matplotlib
#matplotlib.use('Qt4Agg')
import matplotlib.pyplot as plt

### Dask Arrays

Dask arrays are similar to numpy arrays, except that they are chunked into small pieces.

In [3]:
import dask.array as da
from dask.array.image import imread

We can read a collection of images into a dask array.

In [4]:
cell_images = imread(path+"*.jpg")

Let's look at it:

In [5]:
cell_images

dask.array<imread-..., shape=(844, 256, 256, 3), dtype=uint8, chunksize=(1, 256, 256, 3)>

Each image is a separate chunk.

*The images are not loaded into RAM!!!!*

We can do computations on the dask array as long as individual chunks (and the computations on them) fit into RAM.

Let's  find the mean image:

In [ ]:
cell_images.mean(axis=0)

The above statement creates another dask array, which is not loaded into RAM, i.e. it is not calculated yet. 
It waits for us tell dask explicitly when to do the calculation using the `compute` command.This is called [lazy evaluation](https://en.wikipedia.org/wiki/Lazy_evaluation).

In [ ]:
%%time
mean_cell_image = cell_images.mean(axis=0).compute()

In [ ]:
type(mean_cell_image)

In [ ]:
plt.imshow(mean_cell_image)

Well, that is pretty but let's do something more meaningful!

***Important: only the final image is stored in RAM, which is smaller than the original collection!***

Side Note: if we pass a dask array to a function not supporting dask arrays, it often gets converted to an in-memory numpy array and evaluated.

A lot of other functions to perform on images using the [Dask API](http://dask.pydata.org/en/latest/array-api.html).

#### Exercise:
* calculate the mean for each RGB color for each image
* calculate the max for each RGB color for each image
* calculate the min and max across all colors for each image

In [ ]:
RGB_means = cell_images.mean(axis=[1,2])
plt.plot(RGB_means[:,1],RGB_means[:,2],'ro')

In [ ]:
plt.plot(cell_images.max(axis=[1,2])[:,1],'go')

In [ ]:
plt.plot(cell_images.max(axis=[1,2])[:,0],'ro')

In [ ]:
plt.plot(cell_images.max(axis=[1,2])[:,2],'bo')

In [ ]:
plt.plot(cell_images.max(axis=[1,2,3]),'ko')

We observe that some of the images have a very diffent intensity range.

In [ ]:
cell_images[730,:,:,:].max()

In [ ]:
from skimage.exposure import rescale_intensity

In [ ]:
rescale_intensity(np.array(cell_images[700,:,:,:])).max(axis=(0,1))

#### Mean color three coordinates

In [ ]:
plt.plot(image_colors[:,0],image_colors[:,1],'ro')

In [ ]:
image_colors = cell_images.mean(axis = [1,2])

In [ ]:
plt.plot(cell_images.max(axis=[1,2,3]),'ro')

In [ ]:
#average green vs average red 

In [ ]:
green_labels = (image_colors[:,0]<image_colors[:,1])

In [ ]:
plt.figure()
plt.imshow(cell_images[~green_labels,:][6,:,:,:])
plt.show()

---
### Perform Principal Component Analysis on a Collection of Images

PCA can be perform by the following steps:
* center the images
* reshape the data into features x observations format
* perform SVD on the data matrix
* reshape the modes into an image format -> they represent the principal components

In [ ]:
# center the images
cell_images_centered = cell_images - cell_images.mean(axis = 0)

In [ ]:
cell_images_centered

In [ ]:
# create a data matrix - # features x #observations
data = cell_images_centered.reshape((844,256*256*3)).T[:,:1000]
print(data.shape)
data

In [ ]:
# svd expects only one column block
data = data.rechunk((256,100))
data

In [ ]:
u, s, v = da.linalg.svd(data)

In [ ]:
s = s.compute()

In [ ]:
plt.plot(s.compute(),'ro')
plt.show()

In [ ]:
u.shape

In [ ]:
u = u.rechunk((256*256,1)).reshape((256,256,3,100))

In [ ]:
u = u[:,:100].compute()

In [ ]:
u = u.dot(np.diag(s[:100])).reshape(256,256,3,100)

In [ ]:
np.max(u)


In [ ]:
s

In [ ]:
for i in range(u.shape[3]):
    plt.figure()
    plt.imshow(u[:,:,:,i])

In [ ]:
u.shape

----
### Performing complex operations on a stack of images.

We consider the following scenario: 

we have a big pile of images and we need to perform the same pre-processing step to each image and in the end store the result in one array for further processing.

Clearly we can achieve this by writing a `for-loop` which processes each image and stores the result. 

To accomplish this in a distributed manner we can use `dask`'s [delayed](http://dask.pydata.org/en/latest/delayed.html) functionality. It allows to parallelize our own Python functions.

In [ ]:
from dask import delayed

In [ ]:
from skimage import exposure

In [ ]:
rescale_intensity_dask = delayed(rescale_intensity)

In [ ]:
lazy_values = [rescale_intensity_dask(im) for im in cell_images]

In [ ]:
np_image = np.array(cell_images[0,:,:,0])

In [ ]:
arrays = [da.from_delayed(lazy_value, shape=(256,256,3), dtype=np_image.dtype) for lazy_value in lazy_values]

In [ ]:
type(arrays[0])

We can see that the type of this final result is a list and each entry is a `dask array`:

In [ ]:
print(type(arrays))
print(type(arrays[0]))

In [ ]:
arrays

We can convert them to a dask array:

In [ ]:
stack = da.stack(arrays, axis=0) 

In [ ]:
stack

In [ ]:
plt.imshow(stack[6,:,:])

Note: this chunksize is useful when doing individual operations on images, not across images!

In [ ]:
image_colors = stack.mean(axis = [1,2]).compute()

In [ ]:
image_colors.shape

In [ ]:
plt.plot(image_colors[:,2],image_colors[:,0],'ro')

In [ ]:
stack.shape

Now the maximum is always 255.

In [ ]:
plt.plot(stack.max(axis=[1,2,3]),'ro')